In [2]:
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import normalize
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import hdbscan
from sklearn.metrics.pairwise import pairwise_distances


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
# Load LSI vectors and speech IDs
with open("pkl_files/lsi_vectors.pkl", "rb") as f:
    lsi_vectors = pickle.load(f)

with open("pkl_files/speech_ids.pkl", "rb") as f:
    speech_ids = pickle.load(f)

# Normalize LSI vectors
lsi_vectors_normalized = normalize(lsi_vectors)



NameError: name 'normalize' is not defined

# k-means

In [ ]:
# Define the range for number of clusters
cluster_range = range(10, 250, 10)  # From 10 to 100 clusters

# Initialize lists to store evaluation metrics
wcss = []
silhouette_scores = []
# Determine optimal number of clusters using Elbow Method and Silhouette Analysis
for k in cluster_range:
    mbk = KMeans(n_clusters=k, random_state=42, n_init='auto')
    mbk.fit(lsi_vectors_normalized)
    wcss.append(mbk.inertia_)
    
    # Compute silhouette score on a sample
    sample_size = 10000
    sil_score = silhouette_score(lsi_vectors_normalized, mbk.labels_, metric='cosine', sample_size=sample_size, random_state=42)
    silhouette_scores.append(sil_score)
    
    print(f"Processed k={k}, WCSS={mbk.inertia_:.2f}, Silhouette Score={sil_score:.4f}")

In [ ]:
# Plot Elbow Method
plt.figure(figsize=(10,6))
plt.plot(cluster_range, wcss, 'bx-')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.title('Elbow Method For Optimal K')
plt.show()

# Plot Silhouette Scores
plt.figure(figsize=(10,6))
plt.plot(cluster_range, silhouette_scores, 'bx-')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis For Optimal K')
plt.show()

# DB-SCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from sklearn.metrics import silhouette_score

# Step 1: Normalize the dataset
lsi_vectors_normalized = normalize(lsi_vectors)

# Step 2: Define and fit DBSCAN
dbscan_model = DBSCAN(
    eps=0.5,                # Radius for neighborhood (adjust based on data)
    min_samples=10,         # Minimum samples to form a dense cluster
    metric='cosine',        # Use cosine distance for text data
    n_jobs=2           # Use all available CPU cores
)

# Fit the model and get labels
labels = dbscan_model.fit_predict(lsi_vectors_normalized)

# Step 3: Analyze results
# Count the number of clusters (ignoring noise points labeled as -1)
num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
noise_ratio = (labels == -1).sum() / len(labels)

print(f"DBSCAN found {num_clusters} clusters.")
print(f"Noise points: {noise_ratio:.2%}")

# Step 4: Evaluate clustering quality
if num_clusters > 1:
    sil_score = silhouette_score(lsi_vectors_normalized, labels, metric='cosine')
    print(f"Silhouette Score for DBSCAN: {sil_score:.4f}")
else:
    print("Silhouette score not calculated: Only one cluster found.")
